# SQL Subqueries - Lab

## Introduction

Now that you've seen how subqueries work, it's time to get some practice writing them! Not all of the queries will require subqueries, but all will be a bit more complex and require some thought and review about aggregates, grouping, ordering, filtering, joins and subqueries. Good luck!  

## Objectives

You will be able to:

* Write subqueries to decompose complex queries

## CRM Database ERD

Once again, here's the schema for the CRM database you'll continue to practice with.

<img src="https://curriculum-content.s3.amazonaws.com/data-science/images/Database-Schema.png" width="600">

## Connect to the Database

As usual, start by importing the necessary packages and connecting to the database `data.sqlite`.

In [1]:
# Your code here; import the necessary packages
import sqlite3
import pandas as pd
conn = sqlite3.connect('data.sqlite')

In [ ]:
# Your code here; create the connection

## Write an Equivalent Query using a Subquery

The following query works using a `JOIN`. Rewrite it so that it uses a subquery instead.

```
SELECT
    customerNumber,
    contactLastName,
    contactFirstName
FROM customers
JOIN orders 
    USING(customerNumber)
WHERE orderDate = '2003-01-31'
;
```

In [44]:
# Your code here
pd.read_sql("""
SELECT c.customerNumber, c.contactLastName, c.contactFirstName
FROM customers AS c
WHERE customerNumber IN(
SELECT customerNumber
FROM orders
WHERE orderDate = '2003-01-31'

)
""",conn)

,customerNumber,contactLastName,contactFirstName
0,141,Freyre,Diego


## Select the Total Number of Orders for Each Product Name

Sort the results by the total number of items sold for that product.

In [45]:
# Your code here
#two tables products and orders
pd.read_sql("""
SELECT p.productName, od.total_number_of_orders
FROM products p
JOIN (
    SELECT productCode, COUNT(orderNumber) AS total_number_of_orders
    FROM orderdetails
    GROUP BY productCode
) od ON p.productCode = od.productCode
ORDER BY od.total_number_of_orders DESC
""",conn)



,productName,total_number_of_orders
0,1992 Ferrari 360 Spider red,53
1,1969 Harley Davidson Ultimate Chopper,28
2,1952 Alpine Renault 1300,28
3,1996 Moto Guzzi 1100i,28
4,2003 Harley-Davidson Eagle Drag Bike,28
...,...,...
104,1969 Chevrolet Camaro Z28,25
105,2002 Chevy Corvette,25
106,1936 Mercedes Benz 500k Roadster,25
107,1957 Ford Thunderbird,24


## Select the Product Name and the  Total Number of People Who Have Ordered Each Product

Sort the results in descending order.

### A quick note on the SQL  `SELECT DISTINCT` statement:

The `SELECT DISTINCT` statement is used to return only distinct values in the specified column. In other words, it removes the duplicate values in the column from the result set.

Inside a table, a column often contains many duplicate values; and sometimes you only want to list the unique values. If you apply the `DISTINCT` clause to a column that has `NULL`, the `DISTINCT` clause will keep only one NULL and eliminates the other. In other words, the DISTINCT clause treats all `NULL` “values” as the same value.

In [42]:
# Your code here
# Hint: because one of the tables we'll be joining has duplicate customer numbers, you should use DISTINCT
pd.read_sql("""
SELECT pr.productName, total_no_ordered
FROM products AS pr
JOIN (
SELECT productCode ,COUNT(DISTINCT customerNumber) AS total_no_ordered
FROM orderdetails AS od
JOIN orders
ON od.orderNumber = orders.orderNumber
GROUP BY od.productCode
)orderdetails ON orderdetails.productCode = pr.productCode
GROUP BY pr.productName
ORDER BY total_no_ordered DESC

""",conn)


,productName,total_no_ordered
0,1992 Ferrari 360 Spider red,40
1,Boeing X-32A JSF,27
2,1972 Alfa Romeo GTA,27
3,1952 Alpine Renault 1300,27
4,1934 Ford V8 Coupe,27
...,...,...
104,1958 Chevy Corvette Limited Edition,19
105,2002 Chevy Corvette,18
106,1969 Chevrolet Camaro Z28,18
107,1952 Citroen-15CV,18


## Select the Employee Number, First Name, Last Name, City (of the office), and Office Code of the Employees Who Sold Products That Have Been Ordered by Fewer Than 20 people.

This problem is a bit tougher. To start, think about how you might break the problem up. Be sure that your results only list each employee once.

In [59]:
# Your code here
pd.read_sql("""
SELECT e.employeeNumber, e.firstName, e.lastName,  o.officeCode AS office_code, o.city AS city
FROM employees e
JOIN offices o ON e.officeCode = o.officeCode
JOIN customers ON e.employeeNumber = customers.salesRepEmployeeNumber
JOIN orders ON customers.customerNumber = orders.customerNumber
JOIN orderdetails  od ON orders.orderNumber = od.orderNumber
JOIN products pr ON od.productCode = pr.productCode
WHERE pr.productCode IN(
SELECT od.productCode
FROM orderdetails od
GROUP BY od.productCode
HAVING COUNT(DISTINCT od.orderNumber) >1
)
GROUP BY e.employeeNumber, e.firstName, e.lastName, o.city, o.officeCode
ORDER BY e.employeeNumber
""",conn)

,employeeNumber,firstName,lastName,office_code,city
0,1165,Leslie,Jennings,1,San Francisco
1,1166,Leslie,Thompson,1,San Francisco
2,1188,Julie,Firrelli,2,Boston
3,1216,Steve,Patterson,2,Boston
4,1286,Foon Yue,Tseng,3,NYC
5,1323,George,Vanauf,3,NYC
6,1337,Loui,Bondur,4,Paris
7,1370,Gerard,Hernandez,4,Paris
8,1401,Pamela,Castillo,4,Paris
9,1501,Larry,Bott,7,London


## Select the Employee Number, First Name, Last Name, and Number of Customers for Employees Whose Customers Have an Average Credit Limit Over 15K

In [61]:
# Your code here
pd.read_sql("""
SELECT e.employeeNumber, e.firstName, e.lastName, COUNT(c.customerNumber) AS no_of_customers
FROM employees e
JOIN customers c ON e.employeeNumber = c.salesRepEmployeeNumber
GROUP BY e.employeeNumber, e.firstName, e.lastName
HAVING AVG(c.creditLimit) > 15000
ORDER BY no_of_customers DESC, e.lastName, e.firstName
"""
,conn)

,employeeNumber,firstName,lastName,no_of_customers
0,1401,Pamela,Castillo,10
1,1504,Barry,Jones,9
2,1501,Larry,Bott,8
3,1323,George,Vanauf,8
4,1370,Gerard,Hernandez,7
5,1286,Foon Yue,Tseng,7
6,1337,Loui,Bondur,6
7,1188,Julie,Firrelli,6
8,1702,Martin,Gerard,6
9,1165,Leslie,Jennings,6


## Summary

In this lesson, you got to practice some more complex SQL queries, some of which required subqueries. There's still plenty more SQL to be had though; hope you've been enjoying some of these puzzles!